# Load Libraries and Read the File

In [85]:
library(tidyverse) 
library(readr)

In [86]:
data <- read_csv('/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv')
head(data)

Rows: 10000 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Paymen...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NA,2023-03-31


# Convert Columns from Text to Numeric and Date

In [87]:
data <- data %>%
  mutate(
    Quantity = as.numeric(Quantity),
    `Price Per Unit` = as.numeric(`Price Per Unit`),
    `Total Spent` = as.numeric(`Total Spent`),
      `Transaction Date` = as.Date(`Transaction Date`, format = "%Y-%m-%d")
  )

Warning message:
“There were 3 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Quantity = as.numeric(Quantity)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 2 remaining warnings.”


In [88]:
str(data)

tibble [10,000 × 8] (S3: tbl_df/tbl/data.frame)
 $ Transaction ID  : chr [1:10000] "TXN_1961373" "TXN_4977031" "TXN_4271903" "TXN_7034554" ...
 $ Item            : chr [1:10000] "Coffee" "Cake" "Cookie" "Salad" ...
 $ Quantity        : num [1:10000] 2 4 4 2 2 5 3 4 5 5 ...
 $ Price Per Unit  : num [1:10000] 2 3 1 5 2 4 3 4 3 4 ...
 $ Total Spent     : num [1:10000] 4 12 NA 10 4 20 9 16 15 20 ...
 $ Payment Method  : chr [1:10000] "Credit Card" "Cash" "Credit Card" "UNKNOWN" ...
 $ Location        : chr [1:10000] "Takeaway" "In-store" "In-store" "UNKNOWN" ...
 $ Transaction Date: Date[1:10000], format: "2023-09-08" "2023-05-16" ...


# Check for Missing Values

In [89]:
colSums(is.na(data))

Transaction ID             Item         Quantity   Price Per Unit 
               0              333              479              533 
     Total Spent   Payment Method         Location Transaction Date 
             502             2579             3265              460

**Check the Percentage of Missing Values**

In [90]:
colSums(is.na(data)) / nrow(data) * 100

Transaction ID             Item         Quantity   Price Per Unit 
            0.00             3.33             4.79             5.33 
     Total Spent   Payment Method         Location Transaction Date 
            5.02            25.79            32.65             4.60

# Replace Missing Values with Unknown in character Columns

In [91]:
data <- data %>%
  mutate(
    `Payment Method` = replace_na(`Payment Method`, "UNKNOWN"),
    Location = replace_na(Location, "UNKNOWN"),
      Item = replace_na(Item, "UNKNOWN")
  )


In [92]:
colSums(is.na(data))

Transaction ID             Item         Quantity   Price Per Unit 
               0                0              479              533 
     Total Spent   Payment Method         Location Transaction Date 
             502                0                0              460

# Remove Rows with NA in Specific Columns

In [93]:
data <- data %>%
  drop_na(`Transaction Date`)

In [94]:
colSums(is.na(data))

Transaction ID             Item         Quantity   Price Per Unit 
               0                0              454              506 
     Total Spent   Payment Method         Location Transaction Date 
             476                0                0                0

# Replace Missing Values with the Mean in Numeric Columns

In [95]:
data <- data %>%
  mutate(
    Quantity = ifelse(is.na(Quantity), mean(Quantity, na.rm = TRUE), Quantity),
    `Price Per Unit` = ifelse(is.na(`Price Per Unit`), mean(`Price Per Unit`, na.rm = TRUE), `Price Per Unit`),
    `Total Spent` = ifelse(is.na(`Total Spent`), mean(`Total Spent`, na.rm = TRUE), `Total Spent`)
  )


# Replace "ERROR" Values with "Unknown" in Columns

In [96]:

cols_text <- sapply(data, is.character)
cols_numeric <- sapply(data, is.numeric)


cols_with_error_text <- sapply(data[ , cols_text], function(x) any(x == "ERROR"))
cols_with_error_numeric <- sapply(data[ , cols_numeric], function(x) any(x == "ERROR"))


data <- data %>%
  mutate(
    across(.cols = which(cols_text)[cols_with_error_text], .fns = ~replace(., . == "ERROR", "Unknown")),
    across(.cols = which(cols_numeric)[cols_with_error_numeric], .fns = ~replace(., . == "ERROR", "Unknown"))
  )


In [97]:
data <- data %>%
  mutate(
    across(
      .cols = where(is.character),  
      .fns = ~replace(., tolower(.) == "unknown", "Unknown")  
  ))

# Preview and Confirm Changes

In [98]:
colSums(is.na(data))

Transaction ID             Item         Quantity   Price Per Unit 
               0                0                0                0 
     Total Spent   Payment Method         Location Transaction Date 
               0                0                0                0

In [99]:
summary(data)

 Transaction ID         Item              Quantity     Price Per Unit
 Length:9540        Length:9540        Min.   :1.000   Min.   :1.00  
 Class :character   Class :character   1st Qu.:2.000   1st Qu.:2.00  
 Mode  :character   Mode  :character   Median :3.000   Median :3.00  
                                       Mean   :3.025   Mean   :2.95  
                                       3rd Qu.:4.000   3rd Qu.:4.00  
                                       Max.   :5.000   Max.   :5.00  
  Total Spent     Payment Method       Location         Transaction Date    
 Min.   : 1.000   Length:9540        Length:9540        Min.   :2023-01-01  
 1st Qu.: 4.000   Class :character   Class :character   1st Qu.:2023-04-01  
 Median : 8.000   Mode  :character   Mode  :character   Median :2023-07-02  
 Mean   : 8.914                                         Mean   :2023-07-01  
 3rd Qu.:12.000                                         3rd Qu.:2023-10-02  
 Max.   :25.000                                 

In [100]:
head(data)

Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<date>
TXN_1961373,Coffee,2,2,4.000000,Credit Card,Takeaway,2023-09-08
TXN_4977031,Cake,4,3,12.000000,Cash,In-store,2023-05-16
TXN_4271903,Cookie,4,1,8.914442,Credit Card,In-store,2023-07-19
TXN_7034554,Salad,2,5,10.000000,Unknown,Unknown,2023-04-27
TXN_3160411,Coffee,2,2,4.000000,Digital Wallet,In-store,2023-06-11
TXN_2602893,Smoothie,5,4,20.000000,Credit Card,Unknown,2023-03-31


# Save the Cleaned File

In [101]:
write_csv(data, "clean_data.csv")